In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
from tensorflow import keras
import os

# 加载数据

In [18]:
housing = fetch_california_housing()
print(housing.data.shape)
print(housing.target.shape)

(20640, 8)
(20640,)


# 训练集，验证集，测试集

In [19]:
from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,housing.target)
x_train,x_valid,y_train,y_valid = train_test_split(x_train_all,y_train_all)
print('训练数据',x_train.shape,y_train.shape)
print('验证数据',x_valid.shape,y_valid.shape)
print('测试数据',x_test.shape,y_test.shape)

训练数据 (11610, 8) (11610,)
验证数据 (3870, 8) (3870,)
测试数据 (5160, 8) (5160,)


# 标准化处理

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_sacaler = scaler.fit_transform(x_train)
x_valid_sacaler  = scaler.transform(x_valid)
x_test_sacaler  = scaler.transform(x_test)

# 训练

In [15]:
# 遍历数据集
  # 自动求导
  # 评估指标
# epochs 验证集

epochs = 20
batch_size = 32
steps_per_epochs = len(x_train_sacaler) // batch_size
print(steps_per_epochs)
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError() # 均方误差，累积效果
print(metric([5],[2]))
print(metric([0],[1]))
# 不想累积可以重置
metric.reset_state()
metric([5],[2])
print(metric.result())

def random_batch(x,y,batch_size=32):
  idx = np.random.randint(0,len(x),size=batch_size)
  return x[idx],y[idx]

model = keras.models.Sequential([
    input = keras.Layer.Input(shape=x_train_sacaler.shape[1:]),# 8个特征
    keras.layers.Dense(32,activation='rule'),# 32个神经元
    keras.layers.Dense(1)
])

for epoch in range(epochs):
  # 每次epoch重置评估指标
  metric.reset_state()
  for step in range(steps_per_epochs):
    x_batch_data,y_batch_data = random_batch(x_train_sacaler,y_train,batch_size)
    with tf.GradientTape() as tape:
      y_pred = model(x_batch_data) # model是个函数，根据训练数据，得出预测值，w默认有初始化值
      y_pred = tf.squeeze(y_pred,axis=1) # 按照行压缩
      # 计算损失
      loss = keras.losses.MeanSquaredError(y_batch,y_pred) # 计算损失
      metric(y_batch_data,y_pred) # 评估指标
    # 计算梯度
    grad = tape.gradient(loss,model.variables)
    # 更新参数
    grad_and_vars = zip(grad,model.variables)
    optimizer.apply_gradients(grad_and_vars) # 使用优化器更新参数
    print("epoch:",epochs,"train_mse:",metric.result().numpy(),end="")
  y_valid_pred = model(x_valid_sacaler)
  y_valid_pred = tf.squeeze(y_valid_pred,axis=1) # 按照行压缩
  valid_loss = keras.losses.MeanSquaredError(y_valid,y_valid_pred) # 计算损失
  print("\t",'valid mse',valid_loss.numpy())

362
tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(9.0, shape=(), dtype=float32)


# 自定义的效果没有封装的好。封装的是经过优化的
